# Object-oriented programming in Python

Python is multi-purpose and that includes being able to use it for software development. This means it supports object-oriented programming, though there are differences compared to other, statically typed and compiled, languages.

What does object-oriented programming mean exactly?

It means creating:

- **classes** which are *templates* for what attributes and methods an object should have
- objects which are **instances** of those classes with concrete values for the various attributes

To define a class, you need a name (`PascalCase` by convention unlike the rest of Python which is `snake_case`). Technically you could have no other details:

In [1]:
class BankAccount:
    pass

In [2]:
account = BankAccount()
type(account)

__main__.BankAccount

But that's not very interesting.

To define properties in a class, we usually initialise them in the `__init__` method. This is what's called a **magic method** (or "dunder" method because of the double underscores) and it's used to manipulate the internals of an object.

`__init__` is called immediately after an object is created. It is where we can initialise its attributes.

More details about magic methods here: https://rszalski.github.io/magicmethods/ (this was written a while ago and references Python 2 but a lot of the details are still relevant).

Here's a shorter tutorial-style explanation of magic methods: https://www.datacamp.com/tutorial/introducing-python-magic-methods

In [3]:
class BankAccount:
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance

    def print_balance(self):
        print(f"The current balance is {self.balance}")

**Important: the first parameter in a class's method needs to be `self`. It can be any name, but it will refer to the current _instance_ of a class. (Like `this` in other programming languages)**

In [4]:
# we don't need to pass `self`, Python does that for us
# so we just pass in the additional arguments we specified
# in the class definition
account = BankAccount("David", 10_000)

print(account.owner, account.balance)

David 10000


Those are *instance variables* because they differ between `BankAccount` objects.

We can also create *class variables* that have the same value across all instances.

In [5]:
class BankAccount:
    bic_swift = "BANKGB1A"

    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance

    def print_balance(self):
        print(f"The current balance is {self.balance}")

In [6]:
account = BankAccount("David", 10_000)
account.bic_swift

'BANKGB1A'

In [7]:
account_2 = BankAccount("Jeff B", 1_000_000_000)
account_2.bic_swift

'BANKGB1A'

In [8]:
BankAccount.bic_swift

'BANKGB1A'

What's in an object?

In [9]:
account.__dict__

{'owner': 'David', 'balance': 10000}

In [10]:
help(account)

Help on BankAccount in module __main__ object:

class BankAccount(builtins.object)
 |  BankAccount(owner, balance)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, owner, balance)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  print_balance(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  bic_swift = 'BANKGB1A'



Side note: In Python, calling an object's method is the same as calling the method on the class and passing in an instance.

Let's see that in action to clear it up.

This is how to call a method on a class instance:

In [11]:
# `self` is the instance called `account` by default
account.print_balance()

The current balance is 10000


In this alternative usage, we **do** pass in `self` as an argument. You wouldn't see this in practice, it's just how Python functions under the hood:

In [12]:
# here we *have* to specify `self` otherwise we get an error
BankAccount.print_balance(account)

The current balance is 10000


Python has no notion of "private" variables like some other programming languages.

By convention, anything that starts with an underscore should be treated as "private" and not modified directly. Libraries will also warn against changing these internals as they may change in future versions.

In [31]:
class ClassWithPrivateVar:
    def __init__(self):
        self._do_not_touch = 123

    def _secret_method(self):
        return 42

In [32]:
v = ClassWithPrivateVar()

The "private" variable still appears and can be accessed and changed (but it won't appear in autocomplete suggestions):

In [33]:
v.__dict__

{'_do_not_touch': 123}

In [34]:
print(v._do_not_touch)

123


In [35]:
v._do_not_touch = 7

print(v._do_not_touch)

7


In [36]:
v._secret_method()

42

You can try to make something *really* private by using **two** underscores:

In [37]:
class PrivateClass:

    def __init__(self, a):
        self.__a = a

<h1 style="color: #fcd805">Exercise: Creating classes in Python</h1>

1. Create an extended version of the `BankAccount` class, which includes:

- an account number
- a sort code
- a Boolean for whether or not it's a joint account

Create an instance of your class and verify that you can see these new attributes.

2. Create an instance method called `.withdraw` which:

- takes in an amount to withdraw
- if the amount is less than or equal to the current balance, subtract the amount from the balance
- otherwise, print a message to say the withdrawal is not possible
- finally, print the new balance

Create an instance of your class and verify that the `withdraw` function works as intended.

# Magic methods

There are plenty of magic methods that are used under the hood when you use Python objects.

For example printing an object:

In [41]:
print(account)

David's balance is 10000


Actually calls the interal `__str__` method. So if you want to change what happens when an object is "printed" you can redefine this method.

In [42]:
account.__str__()

"David's balance is 10000"

In [25]:
class BankAccount:
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance

    def __str__(self):
        return f"{self.owner}'s balance is {self.balance}"

    def __repr__(self):
        return f"Account: {self.owner=}, {self.balance=}"

    def print_balance(self):
        print(f"The current balance is {self.balance}")

In [44]:
account = BankAccount("David", 10_000)

print(account)

David's balance is 10000


In contrast, when you "look" at a variable in Jupyter (without calling print) it actually calls the `__repr__` method:

In [45]:
account

Account: self.owner='David', self.balance=10000

### Comparisons

Let's say we wanted to tell Python how bank accounts can be compared against each other.

Currently this happens:

In [46]:
account_1 = BankAccount("David", 10_000)
account_2 = BankAccount("Jeff B", 1_000_000_000)

account_1 > account_2

False

Oops. `BankAccount` is a complex class and Python doesn't know how to compare them.

We can fix that!

We just need to define what we mean by:

- less than, using `__lt__`
- less than or equal, using `__le__`
- greater than, using `__gt__`
- greater than or equal, using `__ge__`

In [29]:
class BankAccount:
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance

    def __str__(self):
        return f"{self.owner}'s balance is {self.balance}"

    def __repr__(self):
        return f"Account: {self.owner=}, {self.balance=}"

    # New magic methods to define <, <=, > and >= for bank accounts

    def __lt__(self, other):
        return self.balance < other.balance

    def __le__(self, other):
        return self.balance <= other.balance

    def __gt__(self, other):
        return self.balance > other.balance

    def __ge__(self, other):
        return self.balance >= other.balance

    def print_balance(self):
        print(f"The current balance is {self.balance}")

In [47]:
account_1 = BankAccount("David", 10_000)
account_2 = BankAccount("Jeff B", 1_000_000_000)

print(account_1 < account_2)
print(account_2 <= account_1)
print(account_1 > account_2)
print(account_2 >= account_1)

True
False
False
True


### Encapsulation

There are technically ways to make instance variables readonly (sort of) using magic methods (more here: https://rszalski.github.io/magicmethods/#access).

We can override the `__getattr__` and `__setattr__` methods which are called under the hood when you retrieve or overwrite an attribute.

`__getattr__` gets called when an attribute is not found in an object. This lets you deal with errors in a controlled way.

In [48]:
class ExampleClass:

    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __getattr__(self, name):
        print(f"{name} not found in this object")

In [49]:
eg = ExampleClass(1, 2)

eg.a

1

In [50]:
eg.c

c not found in this object


`__setattr__` gets called whenever you try to set an attribute *regardless of whether it exists*.

In [51]:
class ExampleSetClass:
    def __init__(self, a, b, readonly):
        self.a = a
        self.b = b

        # we can't do self.readonly = readonly because it triggers __setattr__
        # so we modify the internal dict directly
        self.__dict__["readonly"] = readonly

    def __setattr__(self, name, value):
        if name == "readonly":
            print("Cannot modify the readonly attribute!")
        else:
            # you can't do self.name = value
            # because that would call __setattr__ again and cause an infinite loop
            # instead, we manipulate the internal dict:
            self.__dict__[name] = value

In [52]:
eg_2 = ExampleSetClass(1, 2, 123)

eg_2.a

1

In [53]:
eg_2.readonly

123

In [54]:
eg_2.readonly = 1456

Cannot modify the readonly attribute!


In [55]:
eg_2.readonly

123

But even this isn't *really* readonly because we can always turn to magic methods...

In [56]:
eg_2.__dict__["readonly"] = 789

eg_2.readonly

789

<h1 style="color: #fcd805">Exercise: Magic methods in Python</h1>

Here are some requirements for creating a `Loan` class. Use them to code the class and check that all requirements are met.

1. It should contain the following attributes:

- initial loan amount
- remaining loan amount
    - this should not be specified by the user, but set to be the same as the initial loan amount
- interest rate
- term (in years)

2. The initial loan amount, interest rate and term should all be readonly.

3. One `Loan` object is "bigger" than another if the initial loan amount is bigger. Implement all the necessary magic methods to make this work.

4. When printed, a loan object should detail all its attributes: the initial loan amount, the interest rate, the term and what's remaining of the loan.

5. BONUS: find the right magic method and implement it so that when the Python function `len()` is called on a `Loan`, the remaining loan amount is returned.

### Dataclasses

Since Python 3.7, it is possible to create something called a `dataclass`. This is a way to create classes in Python without having a bloated `__init__` method. Instead, we specify all the attributes and their types and everything else just works.

In [57]:
from dataclasses import dataclass

@dataclass
class Mortgage:

    loan_amount: float
    interest_rate: float
    term: int

In [58]:
mortgage = Mortgage(10_000, 2.5, 35)
mortgage

Mortgage(loan_amount=10000, interest_rate=2.5, term=35)

In [59]:
type(mortgage)

__main__.Mortgage

_Note: because Python is dynamically typed, that type information is **not** enforced. They are simply "hints" to the user_

In [60]:
Mortgage("LOAN", ["INTEREST_RATE"], True)

Mortgage(loan_amount='LOAN', interest_rate=['INTEREST_RATE'], term=True)

Similarly, these type "hints" are supported when creating functions.

We can specify what types input variables should be and what the function returns: